In [ ]:
from selenium import webdriver
import urllib.request
# from aws_upload import download_file
# from aws_upload import upload_file
import tempfile
from tqdm import tqdm
import os
import boto3 
# if not os.path.exists('cats'):
#     os.makedirs('cats')
#%%
# Define the driver
ROOT = 'https://all-free-download.com/free-photos/cute-cat-jpg.html'
driver = webdriver.Chrome()
driver.get(ROOT)
s3_client = boto3.client('s3')
# Get the list of cat images
elem = driver.find_element_by_class_name('imgcontainer')
if elem:
    img_list = elem.find_elements_by_class_name('img-responsive')
    if img_list:
        src_list = []
        for src in img_list:
            try:
                src_list.append(src.get_attribute('src'))
            except:
                print('No source found')
        # Create a temporary directory, so you don't store images in your local machine
        with tempfile.TemporaryDirectory() as temp_dir:
            print(temp_dir)
            for i, src in enumerate(tqdm(src_list)):
                urllib.request.urlretrieve(src, f'{temp_dir}/cat_{i}.png')
                s3_client.upload_file(f'{temp_dir}/cat_{i}.png', 'ai-core-bucket-eta-cohort', f'cats/cat_{i}.png')
driver.quit()